In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [ ]:
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# EDA (Exploratory Data Analysis)

In the first DataFrame revision, we can see that it has many columns, making it very difficult to apply EDA to each column individually. Therefore, the approach will be different:

1. Check how the SalesPrice variable behaves.
2. Identify the features that would be most important to me if I were buying a house.
3. Finally, check the correlation between all categorical and numerical features using a heatmap

In [ ]:
df_train.info()

## 1. Sale Price behavior

## 1.1 SalePrice 

min_price = $34.900 , Mean price = $180.921, max_price = $755.000 (outlier maybe) 

In [ ]:
print(df_train['SalePrice'].describe())

In [ ]:
fig = plt.figure(figsize = (15,5))

ax1= plt.subplot(1,2,1)
sns.histplot(df_train, x = 'SalePrice', bins = 'auto', ax = ax1) #Create graph
ax1.set_title('SalesPrice Histogram')

1. It doesn't have a normal distribution
2. It is right skewed

In [ ]:
print(f"The Skewnes  is a measure of asymmetry = {df_train['SalePrice'].skew()}")
print(f"The kurtosis is a measure of the tailedness (high > 3) = {df_train['SalePrice'].kurt()}")

When a distribution has a high kurtosis means that it has a lot of outliers

## 1.2 Sales prices relation with others features

After check all the columns and give them categories and priority, I've chosen 4 features to see how their behave with PriceSales:
1. Bedroom
2. YearBuilt
3. GrLivArea
4. OverallQual

## 2. Important variables

## 2.1 Bedrooms

In [ ]:
df_train['BedroomAbvGr'].value_counts()

In [ ]:
fig = plt.figure(figsize = (10,4))

#Create first graph count per # bedrooms
ax1 = fig.add_subplot(1,2,1)
sns.countplot(df_train, x= 'BedroomAbvGr', ax = ax1, width = 0.5)
ax1.set_title('Count plot # Bedrooms')

#Create second graph average price per # bedrooms
ax2 = fig.add_subplot(1,2,2)
sns.boxplot(df_train, x = 'BedroomAbvGr', y = 'SalePrice', ax = ax2)
ax2.set_title('Spread price # Bedrooms')

plt.tight_layout()  # Adjusts the spacing between the plots

## 2.2 GrLivArea

In [ ]:
var = 'GrLivArea'
df = pd.concat([df_train['SalePrice'],df_train[var]],axis = 1)
df.plot.scatter(x = var, y = 'SalePrice', ylim= (0,800000))

## 2.3 YearBuilt

In [ ]:
# Year built and price box plot
var = 'YearBuilt'
fig = plt.figure(figsize = (30,4))
df = pd.concat([df_train['SalePrice'],df_train[var]], axis = 1)


ax1 = fig.add_subplot(1,2,2)
fig = sns.boxplot(data = df[df['SalePrice'] < 500000], x = var, y = 'SalePrice', ax = ax1)
fig.axis(ymin=0, ymax=500000);
plt.xticks(rotation = 90);

## 2.4 OverallQual

In [ ]:
var = 'OverallQual'
fig = plt.figure(figsize = (25,3))
ax1 = fig.add_subplot(1,2,1)

df = pd.concat([df_train[var],df_train['SalePrice']],axis = 1)
sns.boxplot(data = df[df['SalePrice'] < 500000], x = var, y = 'SalePrice')

## 3. Correlation 

## 3.1 Numerical features correlation

In [ ]:
#Correlation heatmap
df_numeric = df_train.select_dtypes(include = [int,float])
df_numeric = df_numeric.corr()
fig = plt.figure(figsize = (25,12))

ax1 = fig.add_subplot(1,2,1)
sns.heatmap(df_numeric, vmax=.8, square=True, ax = ax1); 

There are several features with a strong correlation between each other, always yearbuilt, overall quality and even basement features

Create a top 10 variables heatmap with the higest SalePrice correlation 

In [ ]:
#Number of variable with the higest correlation
k = 10

#Get the top 10 columns 
top_10 = df_numeric.nlargest(k,['SalePrice'])['SalePrice'].index
df_top_10 = np.corrcoef(df_train[top_10].values.T)
sns.heatmap(df_top_10, annot = True, fmt = '.2g', annot_kws={'size': 10}, yticklabels=top_10.values, xticklabels=top_10.values, square = True)

Some conclusion about this zoom correlation over SalePrice:

1. SalePrice has a strong correlation wit OverallQual, GrLivArea, GarageCars.
2. GarageCars and GarageArea are strongly correlated, but it is logic due to that how many 'GarageCars" depends on 'GarageArea', then both would give us the same information, so we should get just one.
3. Something similar happen with "TotalBsmtSF" and '1stFlrSF", If you have a big basement you will have a big first floor.
4. "TotRmsAbvGrd" and "GrLivArea" same history....
5. YearBuilt and "TotRmsAbvGrd" - "GrLivArea" is a consecuence of the point 4. I mean if a variable has a strong correlation with "TotRmsAbvGrd" for consequence will have a strong correlation with "GrLivArea" as well.

Now create a scatter plot between all the numerical strong correlated variables and sale price

In [ ]:
fig = plt.figure(figsize = (25,5))

# Scatterplot (Pairplot)
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], height=2.5)  # Updated 'size' to 'height'
plt.show()

## Missing data

In [ ]:
total_null = df_train.isna().sum().sort_values(ascending = False)
percentage = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending = False) * 100
df_null = pd.concat([total_null,percentage], axis = 1, keys = ['Total','percent'])
df_null.head(19)

1. We will consider more than 15% is a good limit to remove a column (if this is not an important column).
2. Regarding everyghing related to Garage features, we know that the most important feature is "GarageCars". Therefore we don't need additional garage-related features, and we could delete GarageX.
3. Similarly, for Bsmt features, the most importante feature is "TotalBsmtSF", therefore we could remove the remaining basement related features if they have nulls values.

In [ ]:
columns_remove = df_null[df_null['percent'] >= 0.5].index
df_train = df_train.drop(columns = columns_remove)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isna().sum().max()

## Outliers

In [ ]:
df_train.plot.scatter(x = 'GrLivArea', y= 'SalePrice')

I can see two outliers, both are rare cases, low price for a big place.

In [ ]:
q1, q2, q3 = np.percentile(df_train['SalePrice'],[25,50,75])
iqr = q3 - q1

upper_limit = q3 + (1.5 * iqr)
upper_limit
df_train[df_train['SalePrice'] > upper_limit].plot.scatter(x = 'GrLivArea', y= 'SalePrice')

When using the interquartile range, I identified 61 outliers. However, as we can see, they follow the same pattern or relationship with the sale prices as the regular samples. Therefore, I will remove only the two most extreme outliers from the dataset.

In [ ]:
index_rmv = df_train[(df_train['GrLivArea'] > 4000) & (df_train['SalePrice'] < 300000)].index
df_train = df_train.drop(index_rmv)

## Feature selection

In [ ]:
# Select numerical columns
num_cols = df_train.select_dtypes(include=['int', 'float']).columns

# Drop the 'SalePrice' column
num_cols = num_cols.difference(['SalePrice'])

# Convert to a list if you need a list instead of an Index
num_cols = list(num_cols)

In [ ]:
cat_columns = df_train.select_dtypes(include = ['object']).columns
cat_columns

In [ ]:
features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']

This set of columns was previously selected as having the highest correlation values with SalePrice.

In [ ]:
df_train[num_cols]

## Modeling

In [ ]:
#Defining datasets to train the model
X_train = df_train[num_cols]
Y_train = df_train['SalePrice']
X_test = df_test[num_cols]

## Parameter Tunning

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb

# Set object xgb
xgb = xgb.XGBRegressor()

# Grid Search hyperparameter tuning
param_grid = {
    'max_depth': [3, 5],
    'min_child_weight': [1, 3],
    'reg_alpha': [0, 0.1],
    'n_estimators': [100, 150],
    'learning_rate': [0.05, 0.1],
    'colsample_bytree': [0.8, 1.0]
}

# Execute grid search to get best params
grid_search = GridSearchCV(xgb, param_grid=param_grid, verbose=1, n_jobs=-1)

# Fit model
grid_search.fit(X_train, Y_train,  verbose=False)

#Print best params
print(grid_search.best_params_)

In [ ]:
import xgboost as xgb
#Creating the model with tunned parameters 
regressor = xgb.XGBRegressor(reg_alpha = 0,
              n_estimators = 150,
              min_child_weight = 3,
              max_depth = 3,
              learning_rate = 0.1,
              colsample_bytree = 0.8,
                eval_metric='rmsle')

In [ ]:
regressor2 = xgb.XGBRegressor(learning_rate = grid_search.best_params_["learning_rate"],
                           n_estimators  = grid_search.best_params_["n_estimators"],
                           max_depth     = grid_search.best_params_["max_depth"],
                           eval_metric='rmsle')

regressor.fit(X_train, Y_train,  verbose=False)

#=========================================================================
# To use early_stopping_rounds: 
# "Validation metric needs to improve at least once in every 
# early_stopping_rounds round(s) to continue training."
#=========================================================================
# first perform a test/train split 
#from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)
#regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=False)

#=========================================================================
# use the model to predict the prices for the test data
#=========================================================================
predictions = regressor.predict(X_test)

In [ ]:
df_test[features]

In [ ]:
output = pd.DataFrame({"Id":df_test['Id'], "SalePrice":predictions})
output.to_csv('submission.csv', index=False)
output